In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
# %matplotlib inline 可以在Ipython编译器里直接使用，功能是可以内嵌绘图，并且可以省略掉plt.show()这一步

In [17]:
# 定义超参数
input_size = 28 # 图像的总尺寸
num_classes = 10 # 标签的种类数
num_epochs = 3 # 训练的总循环周期
batch_size = 64

# 训练集
train_dataset = datasets.MNIST(root='./data',
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)

# 测试集
test_dataset = datasets.MNIST(root='./data',
                            train=False,
                            transform=transforms.ToTensor())

# 构建 batch 数据
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                        batch_size=batch_size,
                                        shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                        batch_size=batch_size,
                                        shuffle=True)

$\frac{(h - filterSize + padding * 2)}{stride} + 1$

In [18]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential( # 输入大小(1, 28, 28)
            nn.Conv2d(
                in_channels=1, #灰度图
                out_channels=16, # 得到的特征图个数
                kernel_size=5, # 卷积核大小
                stride=1, # 步长
                padding=2 # 如果希望卷积后大小跟原来一样，需要设置 padding = (kernel_size - 1) / 2 if stride = 1
            ),

            nn.ReLU(), # relu 层
            nn.MaxPool2d(kernel_size=2) # 池化操作（2 x 2 区域），输出结果为 （16， 14， 14）
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, 5, 1, 2), # 输入 （16， 14， 14）
            nn.ReLU(), # relu 层
            nn.MaxPool2d(2) # 输出 （32， 7， 7）
        )
        self.out = nn.Linear(32 * 7 * 7, 10) # 全连接层得到结果

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1) # flatten 操作，结果为：（batch_size, 32 * 7 * 7)
        output = self.out(x)
        return output

准确率作为评估标准
- torch.max(a,1) 返回每一行中最大值的那个元素，且返回其索引
- torch.max()[0]， 只返回最大值的每个数
- troch.max()[1]， 只返回最大值的每个索引

In [19]:
def accuracy(predictions, labels):
    pred = torch.max(predictions.data, 1)[1]
    rights = pred.eq(labels.data.view_as(pred)).sum()
    return rights, len(labels) # 正确预测的结果个数，总的结果个数

训练网络模型

In [21]:
# 实例化
net = CNN()
# 损失函数
criterion = nn.CrossEntropyLoss()
# 优化器
optimizer = optim.Adam(net.parameters(), lr=0.001) # 普通的随机梯度下降算法

# 开始训练循环
for epoch in range(num_epochs):
    # 当前 epoch 的结果保存下来
    train_rights = []

    for batch_idx, (data, target) in enumerate(train_loader): # 针对容器的每一个批进行循环
        net.train()
        output = net(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        right = accuracy(output, target)
        train_rights.append(right)

        if batch_idx % 100 == 0:
            net.eval()
            val_rights = []
            for (data, target) in test_loader:
                output = net(data)
                right = accuracy(output, target)
                val_rights.append(right) # append tuples

            # 准确率计算
            train_r = (sum([tup[0] for tup in train_rights]), sum([tup[1] for tup in train_rights]))
            val_r = (sum([tup[0] for tup in val_rights]), sum([tup[1] for tup in val_rights]))

            print('当前 epoch: {} [{}/{}({:.0f}%)]\t损失: {:.6f}\t训练集准确率: {:.2f}%\t测试集准确率: {:.2f}%'.format(
                epoch,
                batch_idx * batch_size,
                len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.data,
                100. * train_r[0].numpy() / train_r[1],
                100. * val_r[0].numpy() / val_r[1]
            ))






当前 epoch: 0 [0/60000(0%)]	损失: 2.304555	训练集准确率: 9.38%	测试集准确率: 11.77%
当前 epoch: 0 [6400/60000(11%)]	损失: 0.244132	训练集准确率: 75.14%	测试集准确率: 91.42%
当前 epoch: 0 [12800/60000(21%)]	损失: 0.137795	训练集准确率: 84.26%	测试集准确率: 95.14%
当前 epoch: 0 [19200/60000(32%)]	损失: 0.132530	训练集准确率: 87.94%	测试集准确率: 96.48%
当前 epoch: 0 [25600/60000(43%)]	损失: 0.173722	训练集准确率: 89.92%	测试集准确率: 97.01%
当前 epoch: 0 [32000/60000(53%)]	损失: 0.064016	训练集准确率: 91.27%	测试集准确率: 97.17%
当前 epoch: 0 [38400/60000(64%)]	损失: 0.020512	训练集准确率: 92.27%	测试集准确率: 97.66%
当前 epoch: 0 [44800/60000(75%)]	损失: 0.122267	训练集准确率: 93.01%	测试集准确率: 97.83%
当前 epoch: 0 [51200/60000(85%)]	损失: 0.061263	训练集准确率: 93.57%	测试集准确率: 97.83%
当前 epoch: 0 [57600/60000(96%)]	损失: 0.123808	训练集准确率: 94.04%	测试集准确率: 98.20%
当前 epoch: 1 [0/60000(0%)]	损失: 0.107763	训练集准确率: 98.44%	测试集准确率: 98.25%
当前 epoch: 1 [6400/60000(11%)]	损失: 0.069549	训练集准确率: 98.05%	测试集准确率: 98.18%
当前 epoch: 1 [12800/60000(21%)]	损失: 0.054784	训练集准确率: 97.88%	测试集准确率: 98.42%
当前 epoch: 1 [19200/60000(32%)]	损失: 0.076677	训练集准确率: